import packages

In [ ]:
import pandas as pd
from pandas.io.json import json_normalize
import requests
import json
import time

create dictionaries to be used to map ID values in dataframe to meaningful play descriptions. NOTE: these are incomplete and need to be fleshed out.

In [ ]:
event_types = {
    1: 'made shot',
    2: 'missed shot',
    3: 'free throw',
    4: 'rebound',
    5: 'turnover',
    6: 'personal foul',
    8: 'substitution',
    9: 'timeout',
    10: 'jump ball',
    12: 'start period',
    13: 'end period',
    18: 'instant replay',
    20: 'stoppage'
}

opt1_types = {
    3: '3_pt'
}

etypes = {
    1: 'made',
    2: 'missed'
}

mtypes = {
    5: 'layup',
    6: 'driving layup',
    # 7 appears to include dunks and discontinued dribbles?
    9: 'driving dunk',
    # 11 is a ton of stuff, but mostly 1st turnovers?
    # 12 is mostly 2nd turnovers
    13: 'third free throw',
    58: 'turnaround hook'
}

retrieve the list of players and teams for the season from the NBA API

In [ ]:
player_endpoint_url = "http://data.nba.net/10s/prod/v1/2016/players.json"
players_json = json.loads(requests.get(player_endpoint_url).content)
teams_endpoint_url = "http://data.nba.net/10s/prod/v1/2016/teams.json"
teams_json = json.loads(requests.get(teams_endpoint_url).content)

turn the player data into tables

In [ ]:
# select just the players that played during the regular season, not summer league or preseason nonsense
raw_players = json_normalize(players_json['league']['standard'])
# get a subset of the player data to joing to pbp tables
if 'temporaryDisplayName' in list(raw_players.columns):
    player_names = raw_players[['personId', 'temporaryDisplayName']]
else:
    player_names = raw_players[['personId', 'firstName', 'lastName']]
    

turn team data into tables

In [ ]:
# get just the teams from the regular season
raw_teams = json_normalize(teams_json['league']['standard'])
# subset the team names to join back to play-by-play data
team_names = raw_teams[['fullName', 'teamId']]

create function that takes a game ID and returns a dataframe of play-by-play data. Note that this intentionally returns a very raw copy of the dataframe, the idea being we can save the raw data to flat files/pickle files and reprocess them with additional fields and mappings as needed as we figure out more ways into the day. Hit the API once, process later as many times as needed. 

In [ ]:
def retrieve_pbp_df(game_id, year = '2019'):
    # construct the url
    game_url = "https://data.nba.com/data/10s/v2015/json/mobile_teams/nba/" + year + "/scores/pbp/" + game_id + "_full_pbp.json"
    # make the API call and retreive the contents as a string
    try:
        pbp_raw_content = requests.get(game_url).content
        # extract the bytestring into a json object (a python dict)
        pbp_json = json.loads(pbp_raw_content)
        # crete a list of dataframes from the raw pbp data
        #   the g.pd is a list of game periods
        #   we call json_normalize to turn each list of play actions in a period into a dataframe
        pbp_periods = [json_normalize(quarter['pla']) for quarter in pbp_json['g']['pd']]
        # concat the periods together as one dataframe
        play_by_play_raw = (pd.concat(pbp_periods)
                            # make sure the player ID gets stored as a string- there's no need
                            #   to save it as a number and it messes with later joins where the
                            #   type is object because not all elements are populated
                              .assign(pid = lambda df: df.pid.astype(str),
                            # append the game id onto the dataframe
                                      game_id = game_id)
                           )
        # sleep for a few seconds to rate limit how often we hit the API
        time.sleep(2)
        return play_by_play_raw
    except Exception as e:
        print(game_url)
        print(e)
    return None

create lists of game IDs to retrieve.

Format: 00(last 2 digits of starting season)(number of game, ordered by start time of season)
ex. 00190001 is the first game of the 2019-2020 season, 00190500 is the 500th game of the 2019-2020 season.

retrieve most recent ~50 games

In [ ]:
game_ids1 = [str(n).zfill(10) for n in range(21500001,21500100)]
game_dfs1 = [retrieve_pbp_df(game_id, '2015') for game_id in game_ids1]

In [ ]:
time.sleep(90)

In [ ]:
game_ids2 = [str(n).zfill(10) for n in range(21600101,21600200)]
game_dfs2 = [retrieve_pbp_df(game_id, '2016') for game_id in game_ids2]

In [ ]:
time.sleep(90)

In [ ]:
game_ids3 = [str(n).zfill(10) for n in range(21600201,21600300)]
game_dfs3 = [retrieve_pbp_df(game_id, '2016') for game_id in game_ids3]

In [ ]:
time.sleep(90)

In [ ]:
game_ids4 = [str(n).zfill(10) for n in range(21600301,21600400)]
game_dfs4 = [retrieve_pbp_df(game_id, '2016') for game_id in game_ids4]

In [ ]:
time.sleep(90)

In [ ]:
game_ids5 = [str(n).zfill(10) for n in range(21600401,21600500)]
game_dfs5 = [retrieve_pbp_df(game_id, '2016') for game_id in game_ids5]

In [ ]:
time.sleep(90)

In [ ]:
game_ids6 = [str(n).zfill(10) for n in range(21600501,21600600)]
game_dfs6 = [retrieve_pbp_df(game_id, '2016') for game_id in game_ids6]

In [ ]:
time.sleep(90)

In [ ]:
game_ids7 = [str(n).zfill(10) for n in range(21600601,21600700)]
game_dfs7 = [retrieve_pbp_df(game_id, '2016') for game_id in game_ids7]

In [ ]:
time.sleep(90)

In [ ]:
game_ids8 = [str(n).zfill(10) for n in range(21600701,21600800)]
game_dfs8 = [retrieve_pbp_df(game_id, '2016') for game_id in game_ids8]

In [ ]:
time.sleep(90)

In [ ]:
game_ids9 = [str(n).zfill(10) for n in range(21600801,21600900)]
game_dfs9 = [retrieve_pbp_df(game_id, '2016') for game_id in game_ids9]

In [ ]:
time.sleep(90)

In [ ]:
game_ids10 = [str(n).zfill(10) for n in range(21600901,21601000)]
game_dfs10 = [retrieve_pbp_df(game_id, '2016') for game_id in game_ids9]

In [ ]:
time.sleep(90)

In [ ]:
game_ids11 = [str(n).zfill(10) for n in range(21601001,21601230)]
game_dfs11 = [retrieve_pbp_df(game_id, '2016') for game_id in game_ids11]

In [ ]:
all_games = [game_dfs1, game_dfs2, game_dfs3, game_dfs4, game_dfs5, game_dfs6, game_dfs7, game_dfs8, game_dfs9, game_dfs10, game_dfs11]

In [ ]:
all_game_rows = pd.concat([pd.concat(x) for x in all_games])
all_game_rows.shape

In [ ]:
all_game_rows

In [ ]:
all_game_rows.to_csv('2016_pbp.csv', index=False)

modify the below cell if you would like to apply the field mapping to a differnt subset of games.

In [ ]:
play_by_play_raw = all_game_rows

merge the player names onto the dataset, joining on the player ID fields. Same logic could be done for team ID fields, just didn't get that far.

In [ ]:
play_by_play_before_maps = (play_by_play_raw
 .merge(player_names
            .rename({
                'personId':'opid',
                'temporaryDisplayName': 'opName'},
                axis='columns'),
        how = 'left', 
        on='opid')
 .merge(player_names
            .rename({
                'personId': 'pid',
                'temporaryDisplayName': 'pName'},
                axis='columns'),
       how = 'left',
       on = 'pid')
 .merge(player_names
           .rename({
               'personId': 'epid',
               'temporaryDisplayName': 'epName'},
               axis='columns'),
       how='left',
       on='epid')

play_by_play_before_maps.head(5)

add additional fields based on the dictionary mappings above

In [ ]:
play_by_play_with_maps = (play_by_play_before_maps
 .assign(event_type = lambda df: df.etype.map(event_types),
         opt1_type = lambda df: df.opt1.map(opt1_types),
         etype_type = lambda df: df.etype.map(etypes),
         mtypes = lambda df: df.mtype.map(mtypes)))
play_by_play_with_maps

Old single-game code used to test logic before slamming the API

In [ ]:
sample_pbp_url = "https://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2019/scores/pbp/0021900958_full_pbp.json"
pbp_json = json.loads(requests.get(sample_pbp_url).content)
#json_normalize(pbp_json['g']['pd'])
x = [json_normalize(quarter['pla']) for quarter in pbp_json['g']['pd']]
play_by_play_raw = pd.concat(x).assign(pid = lambda df: df.pid.astype(str))
play_by_play_raw.head()